## One hot models
include one-hot CNN and one-hot Residual models

In [1]:
import tensorflow as tf
import h5py 
import scipy.stats
import numpy as np
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype_list = ['K562','HepG2']

2023-05-16 00:44:57.635886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 00:44:58.211346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
tf.get_logger().setLevel('ERROR')
for celltype in celltype_list:
    cnn_dir = '/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/base_CNN_model.h5'
    rb_dir = '/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/model.h5'
    mprann_dir = '/home/ztang/multitask_RNA/model_rep/MPRAnn/'+celltype+'/model_0.h5'
    print('###########' + celltype + '########')
    exp_df = pd.read_csv('/home/ztang/multitask_RNA/data/CAGI/'+celltype+'/metadata.csv')
    target = exp_df['6'].values.tolist()
    
    file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/"+celltype+"/onehot.h5", "r")
    alt = file['alt']
    ref = file['ref']

    for model_dir in [cnn_dir,rb_dir,mprann_dir]:
        print(model_dir.split('/')[-1])
        model = tf.keras.models.load_model(model_dir)
        alt_pred = model.predict(alt,verbose=0)
        ref_pred = model.predict(ref,verbose = 0)
        pred = alt_pred - ref_pred
        perf = {}
        for exp in exp_df['8'].unique():
            sub_df = exp_df[exp_df['8'] == exp]
            exp_target = np.array(target)[sub_df.index.to_list()]
            exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
            perf[exp] = scipy.stats.pearsonr(exp_pred,exp_target)[0]
        print(perf)
        print(np.mean(list(perf.values())))



###########K562########
base_CNN_model.h5


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


{'PKLR': 0.42567737884642554}
0.42567737884642554
model.h5
{'PKLR': 0.5509142238286401}
0.5509142238286401
model_0.h5
{'PKLR': 0.36862601391068034}
0.36862601391068034
###########HepG2########
base_CNN_model.h5


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


{'LDLR': 0.3140139515408628, 'SORT1': 0.26455503079262954, 'F9': 0.35978456128820147}
0.3127845145405646
model.h5
{'LDLR': 0.5142895208051962, 'SORT1': 0.4411468053560604, 'F9': 0.5025298928547497}
0.4859887396720021
model_0.h5
{'LDLR': 0.4117558009709818, 'SORT1': 0.20156550190800085, 'F9': 0.2907904992830292}
0.3013706007206706


## Embedding models
GPN embedding trained models

In [1]:
import tensorflow as tf
import h5py 
import scipy.stats
import numpy as np
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype_list = ['K562','HepG2']

2023-05-18 14:55:00.195360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 14:55:00.877993: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
for celltype in celltype_list:
    print('######'+celltype+'######')
    #Load model and dataset
    model_dir = '/home/ztang/multitask_RNA/model/lenti_MPRA_embed/'+celltype+'/gpn.h5'
    data = '/home/ztang/multitask_RNA/data/CAGI_embed/'+celltype+'/gpn.h5'
    file = h5py.File(data,'r')
    model = tf.keras.models.load_model(model_dir)
    
    #Read CAGI metadata
    exp_df = pd.read_csv('/home/ztang/multitask_RNA/data/CAGI/'+celltype+'/metadata.csv')
    target = exp_df['6'].values.tolist()

    #zero-shot predictions
    alt_pred = model.predict(file['alt'],verbose = 0)
    ref_pred = model.predict(file['ref'],verbose=0)
    pred = alt_pred - ref_pred

    #evaluations
    perf = {}
    for exp in exp_df['8'].unique():
        sub_df = exp_df[exp_df['8'] == exp]
        exp_target = np.array(target)[sub_df.index.to_list()]
        exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
        perf[exp] = scipy.stats.pearsonr(exp_pred,exp_target)[0]
    print(perf)
    print(np.mean(list(perf.values())))

######K562######


2023-05-18 14:55:02.883370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78865 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:07:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2023-05-18 14:55:04.396765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-18 14:55:04.495357: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


{'PKLR': 0.4567031168164604}
0.4567031168164604
######HepG2######


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


{'LDLR': 0.4163374859338064, 'SORT1': 0.2790767128295285, 'F9': 0.4366837545072331}
0.37736598442352265


In [3]:
file.keys()

<KeysViewHDF5 ['alt', 'ref']>